In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
k = 4
s = -1/(k-1)*30
C_num = 3

In [3]:
file_data = 'timit_data_train_soap_0211.csv'
data = pd.read_csv(file_data)
my_data = data.values

my_x = my_data[:,1:(len(my_data[0,:]))]
n = len(my_x[:,0])
x_dim = len(my_x[0,:])

file_data = 'timit_y_train_soap_0211.csv'
data = pd.read_csv(file_data)
my_data = data.values

my_y = my_data[:,1] - 1 ##注意减一
my_y = my_y.reshape((len(my_y),1))

In [4]:
file_data = 'timit_data_test_soap_0211.csv'
data = pd.read_csv(file_data)
my_data = data.values

my_x_test = my_data[:,1:(len(my_data[0,:]))]
n_test = len(my_x_test[:,0])
x_dim = len(my_x_test[0,:])

file_data = 'timit_y_test_soap_0211.csv'
data = pd.read_csv(file_data)
my_data = data.values

my_y_test = my_data[:,1] - 1 ##注意减一
my_y_test = my_y_test.reshape((len(my_y_test),1))

In [5]:
print(my_x[0,:])

[8.20093221 0.54       0.1        0.24      ]


In [10]:
print(len(my_y))
print(len(my_y_test))

156
85


In [12]:
# if just use functional data
my_x = my_x[:,0:2]
my_x_test = my_x_test[:,0:2]
x_dim = 2

In [13]:
my_omega = np.random.randn(x_dim,k)
my_b = np.random.randn(k)
C_num = 0.2
correct_num = 0
iter_num = 200

train_s_cr = []
test_s_cr = []

for i in range(n):
    esti = np.dot(my_x[i,:], my_omega) + my_b
    if esti.argmax() == my_y[i]:
        correct_num += 1
print("初始准确率:",correct_num/n)

for iter in range(iter_num):
    my_beta = np.zeros((n,k))
    ##### beta
    for i in range(n):
        pred_iter = (np.dot(my_x[i,:], my_omega) + my_b)
        pred_copy = pred_iter.copy()
        pred_iter[int(my_y[i])] = -float('inf')
        j = pred_iter.argmax()

        if((pred_copy[int(my_y[i])] - pred_copy[j]) < s):
            my_beta[i,j] = C_num
    
    ##### function1
    my_alpha = cp.Variable((n,k))
    obj_function = 0
    
    s1 = 0
    for j in range(k):
        s1_1 = 0
        s1_2 = 0
        for i in range(n):
            if my_y[i] == j:
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s1_1 += (my_alpha[i,j2] - my_beta[i,j2]) *  my_x[i,:]

            if my_y[i] != j:
                s1_2 += (my_alpha[i,j] - my_beta[i,j]) *  my_x[i,:]

        s1 += cp.sum_squares(s1_1 - s1_2)
        
    s2 = 0
    for i in range(n):
        for j2 in range(k):
            if j2 != my_y[i]:
                s2 += my_alpha[i,j2]
                
    obj_function = 0.5 * s1 - s2
    
    ##### constrains1
    constrains = []
    for j in range(k):
        s1_1 = 0
        s1_2 = 0
        for i in range(n):
            if my_y[i] == j:
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s1_1 += (my_alpha[i,j2] - my_beta[i,j2])

            if my_y[i] != j:
                s1_2 += (my_alpha[i,j] - my_beta[i,j])

        constrains += [(s1_1 - s1_2) == 0]

    for i in range(n):
        s1 = 0
        for j in range(k):
            if j != my_y[i]:
                s1 += my_alpha[i,j]
                constrains += [s1 >= 0]
                constrains += [s1 <= C_num]

    for i in range(n):
        for j in range(k):
            if j != my_y[i]:
                constrains += [my_alpha[i,j] >= 0]
                
    prob = cp.Problem(cp.Minimize(obj_function), constrains)
    prob.solve()
    
    ##### calculate omega
    for j in range(k):
        s1_1 = 0
        s1_2 = 0
        for i in range(n):
            if my_y[i] == j:
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s1_1 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

            if my_y[i] != j:
                s1_2 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

        my_omega[:,j] = s1_1 - s1_2
        
    #print(my_omega)
    
    #####function2
    my_eta = cp.Variable(n)
    my_b = cp.Variable(k)
    
    obj_function_2 = 0
    s1 = 0
    for j in range(k):
        s1_1 = 0
        s1_2 = 0
        for i in range(n):
            if my_y[i] == j:
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s1_1 += my_beta[i,j2]

            if my_y[i] != j:
                s1_2 += my_beta[i,j2]

        s1 += (s1_1 - s1_2) * my_b[j]

    obj_function_2 = C_num * cp.sum(my_eta) + s1
    
    ##### constrains2
    
    constrains_2 = []
    f_sim = np.dot(my_x, my_omega)

    for i in range(n):
        constrains_2 += [my_eta[i] >= 0]

    for i in range(n):
        for j in range(k):
            if j != my_y[i]:
                constrains_2 += [my_eta[i] >= (1 - (f_sim[i,int(my_y[i])] + my_b[int(my_y[i])]) + f_sim[i,j] + my_b[j])]

    constrains_2 += [cp.sum(my_b) == 0]
    
    prob_2 = cp.Problem(cp.Minimize(obj_function_2), constrains_2)
    
    prob_2.solve()
    my_b = my_b.value
    #print(my_b)
    
    ### train set correct-ratio ###
    
    correct_num = 0
    esti_mat = np.zeros(n)
    for i in range(n):
        esti = np.dot(my_x[i,:], my_omega) + my_b
        esti_mat[i] = esti.argmax()
        if esti.argmax() == my_y[i]:
            correct_num += 1
    #print(correct_num/n)
    
    cf_mat = np.zeros((k,k))
    for i in range(n):
        cf_mat[int(my_y[i]),int(esti_mat[i])] += 1
    #print(cf_mat)
    print("iter:{},准确率:{}".format(iter, correct_num/n))
    print(cf_mat)
    train_s_cr.append(correct_num/n)
    
    
    ### test set correct-ratio ###
    correct_num = 0
    esti_mat_test = np.zeros(n_test)
    for i in range(n_test):
        esti = np.dot(my_x_test[i,:], my_omega) + my_b
        esti_mat_test[i] = esti.argmax()
        if esti.argmax() == my_y_test[i]:
            correct_num += 1
    #print(correct_num/n)

    cf_mat = np.zeros((k,k))
    for i in range(n_test):
        cf_mat[int(my_y_test[i]),int(esti_mat_test[i])] += 1
    #print(cf_mat)
    print("准确率:{}".format(correct_num/n_test))
    print(cf_mat)
    test_s_cr.append(correct_num/n_test)

初始准确率: 0.2692307692307692
iter:0,准确率:0.3076923076923077
[[13. 17.  1. 12.]
 [ 3. 23.  4. 23.]
 [ 8. 11.  0.  1.]
 [16. 11.  1. 12.]]
准确率:0.23529411764705882
[[ 7.  7.  2.  4.]
 [ 2.  6.  0. 13.]
 [ 6.  4.  1.  7.]
 [16.  4.  0.  6.]]
iter:1,准确率:0.3269230769230769
[[14. 18.  0. 11.]
 [ 4. 26.  4. 19.]
 [ 8. 11.  0.  1.]
 [17. 12.  0. 11.]]
准确率:0.24705882352941178
[[ 7.  9.  0.  4.]
 [ 3.  7.  0. 11.]
 [ 6.  4.  1.  7.]
 [16.  4.  0.  6.]]
iter:2,准确率:0.3717948717948718
[[13. 21.  0.  9.]
 [ 2. 35.  0. 16.]
 [ 8. 11.  0.  1.]
 [15. 15.  0. 10.]]
准确率:0.25882352941176473
[[ 7. 11.  0.  2.]
 [ 2.  9.  0. 10.]
 [ 6.  6.  0.  6.]
 [16.  4.  0.  6.]]
iter:3,准确率:0.40384615384615385
[[13. 18.  4.  8.]
 [ 2. 35.  2. 14.]
 [ 8.  6.  6.  0.]
 [15.  9.  7.  9.]]
准确率:0.29411764705882354
[[ 7.  9.  3.  1.]
 [ 2. 11.  0.  8.]
 [ 6.  4.  2.  6.]
 [16.  4.  1.  5.]]
iter:4,准确率:0.38461538461538464
[[13. 23.  0.  7.]
 [ 0. 39.  0. 14.]
 [ 8. 12.  0.  0.]
 [15. 17.  0.  8.]]
准确率:0.25882352941176473
[[ 7. 12.

iter:40,准确率:0.3782051282051282
[[13. 22.  0.  8.]
 [ 1. 37.  0. 15.]
 [ 8. 12.  0.  0.]
 [15. 16.  0.  9.]]
准确率:0.25882352941176473
[[ 7. 12.  0.  1.]
 [ 2. 10.  0.  9.]
 [ 6.  6.  0.  6.]
 [16.  5.  0.  5.]]
iter:41,准确率:0.41025641025641024
[[13. 19.  4.  7.]
 [ 1. 37.  3. 12.]
 [ 8.  6.  6.  0.]
 [15. 10.  7.  8.]]
准确率:0.2823529411764706
[[ 7.  9.  3.  1.]
 [ 2. 11.  0.  8.]
 [ 6.  6.  2.  4.]
 [16.  5.  1.  4.]]
iter:42,准确率:0.3782051282051282
[[13. 22.  0.  8.]
 [ 1. 37.  0. 15.]
 [ 8. 12.  0.  0.]
 [15. 16.  0.  9.]]
准确率:0.25882352941176473
[[ 7. 12.  0.  1.]
 [ 2. 10.  0.  9.]
 [ 6.  6.  0.  6.]
 [16.  5.  0.  5.]]
iter:43,准确率:0.41025641025641024
[[13. 19.  4.  7.]
 [ 1. 37.  3. 12.]
 [ 8.  6.  6.  0.]
 [15. 10.  7.  8.]]
准确率:0.2823529411764706
[[ 7.  9.  3.  1.]
 [ 2. 11.  0.  8.]
 [ 6.  6.  2.  4.]
 [16.  5.  1.  4.]]
iter:44,准确率:0.3782051282051282
[[13. 22.  0.  8.]
 [ 1. 37.  0. 15.]
 [ 8. 12.  0.  0.]
 [15. 16.  0.  9.]]
准确率:0.25882352941176473
[[ 7. 12.  0.  1.]
 [ 2. 10.  0.

iter:80,准确率:0.3782051282051282
[[13. 22.  0.  8.]
 [ 1. 37.  0. 15.]
 [ 8. 12.  0.  0.]
 [15. 16.  0.  9.]]
准确率:0.25882352941176473
[[ 7. 12.  0.  1.]
 [ 2. 10.  0.  9.]
 [ 6.  6.  0.  6.]
 [16.  5.  0.  5.]]
iter:81,准确率:0.41025641025641024
[[13. 19.  4.  7.]
 [ 1. 37.  3. 12.]
 [ 8.  6.  6.  0.]
 [15. 10.  7.  8.]]
准确率:0.2823529411764706
[[ 7.  9.  3.  1.]
 [ 2. 11.  0.  8.]
 [ 6.  6.  2.  4.]
 [16.  5.  1.  4.]]
iter:82,准确率:0.3782051282051282
[[13. 22.  0.  8.]
 [ 1. 37.  0. 15.]
 [ 8. 12.  0.  0.]
 [15. 16.  0.  9.]]
准确率:0.25882352941176473
[[ 7. 12.  0.  1.]
 [ 2. 10.  0.  9.]
 [ 6.  6.  0.  6.]
 [16.  5.  0.  5.]]
iter:83,准确率:0.41025641025641024
[[13. 19.  4.  7.]
 [ 1. 37.  3. 12.]
 [ 8.  6.  6.  0.]
 [15. 10.  7.  8.]]
准确率:0.2823529411764706
[[ 7.  9.  3.  1.]
 [ 2. 11.  0.  8.]
 [ 6.  6.  2.  4.]
 [16.  5.  1.  4.]]
iter:84,准确率:0.3782051282051282
[[13. 22.  0.  8.]
 [ 1. 37.  0. 15.]
 [ 8. 12.  0.  0.]
 [15. 16.  0.  9.]]
准确率:0.25882352941176473
[[ 7. 12.  0.  1.]
 [ 2. 10.  0.

iter:120,准确率:0.3782051282051282
[[13. 22.  0.  8.]
 [ 1. 37.  0. 15.]
 [ 8. 12.  0.  0.]
 [15. 16.  0.  9.]]
准确率:0.25882352941176473
[[ 7. 12.  0.  1.]
 [ 2. 10.  0.  9.]
 [ 6.  6.  0.  6.]
 [16.  5.  0.  5.]]
iter:121,准确率:0.41025641025641024
[[13. 19.  4.  7.]
 [ 1. 37.  3. 12.]
 [ 8.  6.  6.  0.]
 [15. 10.  7.  8.]]
准确率:0.2823529411764706
[[ 7.  9.  3.  1.]
 [ 2. 11.  0.  8.]
 [ 6.  6.  2.  4.]
 [16.  5.  1.  4.]]
iter:122,准确率:0.3782051282051282
[[13. 22.  0.  8.]
 [ 1. 37.  0. 15.]
 [ 8. 12.  0.  0.]
 [15. 16.  0.  9.]]
准确率:0.25882352941176473
[[ 7. 12.  0.  1.]
 [ 2. 10.  0.  9.]
 [ 6.  6.  0.  6.]
 [16.  5.  0.  5.]]
iter:123,准确率:0.41025641025641024
[[13. 19.  4.  7.]
 [ 1. 37.  3. 12.]
 [ 8.  6.  6.  0.]
 [15. 10.  7.  8.]]
准确率:0.2823529411764706
[[ 7.  9.  3.  1.]
 [ 2. 11.  0.  8.]
 [ 6.  6.  2.  4.]
 [16.  5.  1.  4.]]
iter:124,准确率:0.3782051282051282
[[13. 22.  0.  8.]
 [ 1. 37.  0. 15.]
 [ 8. 12.  0.  0.]
 [15. 16.  0.  9.]]
准确率:0.25882352941176473
[[ 7. 12.  0.  1.]
 [ 2. 10

iter:160,准确率:0.3782051282051282
[[13. 22.  0.  8.]
 [ 1. 37.  0. 15.]
 [ 8. 12.  0.  0.]
 [15. 16.  0.  9.]]
准确率:0.25882352941176473
[[ 7. 12.  0.  1.]
 [ 2. 10.  0.  9.]
 [ 6.  6.  0.  6.]
 [16.  5.  0.  5.]]
iter:161,准确率:0.41025641025641024
[[13. 19.  4.  7.]
 [ 1. 37.  3. 12.]
 [ 8.  6.  6.  0.]
 [15. 10.  7.  8.]]
准确率:0.2823529411764706
[[ 7.  9.  3.  1.]
 [ 2. 11.  0.  8.]
 [ 6.  6.  2.  4.]
 [16.  5.  1.  4.]]
iter:162,准确率:0.3782051282051282
[[13. 22.  0.  8.]
 [ 1. 37.  0. 15.]
 [ 8. 12.  0.  0.]
 [15. 16.  0.  9.]]
准确率:0.25882352941176473
[[ 7. 12.  0.  1.]
 [ 2. 10.  0.  9.]
 [ 6.  6.  0.  6.]
 [16.  5.  0.  5.]]
iter:163,准确率:0.41025641025641024
[[13. 19.  4.  7.]
 [ 1. 37.  3. 12.]
 [ 8.  6.  6.  0.]
 [15. 10.  7.  8.]]
准确率:0.2823529411764706
[[ 7.  9.  3.  1.]
 [ 2. 11.  0.  8.]
 [ 6.  6.  2.  4.]
 [16.  5.  1.  4.]]
iter:164,准确率:0.3782051282051282
[[13. 22.  0.  8.]
 [ 1. 37.  0. 15.]
 [ 8. 12.  0.  0.]
 [15. 16.  0.  9.]]
准确率:0.25882352941176473
[[ 7. 12.  0.  1.]
 [ 2. 10

In [ ]:
x_line = np.linspace(1,iter_num,iter_num)
plt.plot(x_line, train_s_cr)
plt.plot(x_line, test_s_cr)

plt.legend(['train_s_cr','test_s_cr'])
plt.show()